# Разработка модели для классифицикации комментариев на позитивные и негативные

**Цель работы:**<br>
Построить модель, которая определит тональность комментариев в новом сервисе закачика.

**Планируемое использование результата работы:**<br>
Заказчику необходим инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

**Входные данные:**<br>
Заказчик предоставил набор данных с разметкой о токсичности. Негативный комментарий - класс 1, нейтральный комментарий - класс 0.

**Задачи работы:**
1. Подготовить данные для формирования модели.
2. Разработать несколько моделей машинного обучения (МО) для предсказания.
3. Оценить качество работы моделей. Выбрать лучшую модель.
4. Протестировать лучшую модель.

**Дополнительные критерии заказчика:**

- Значение метрики F1 на тестовой выборке должно быть не мешьше 0.75.

**План работы:**
<a id='begin'></a>

1. Шаг 1. Подготовка данных.
    - [[Перейти к разделу]](#step11). [[Перейти к выводу]](#conclusion11). Загрузка и зучение данных. Изучить входные данные, оценить полноту и качество входных данных для достижения цели исследования. Определить задачи предобработки данных.
    - [[Перейти к разделу]](#step12). [[Перейти к выводу]](#conclusion12). Преобработка данных. Осуществить предобработку данных в части улучшения качества данных для дальнейшего анализа.
    - [[Перейти к разделу]](#step13). [[Перейти к выводу]](#conclusion13). Исследовательский анализ данных. Провести исследовательский анализ данных, оценить (уточнить) необходимость формирования дополнительных категорий, параметров и групп данных для достижения цели исследования, провести корреляционный анализ данных.
    - [[Перейти к разделу]](#step14). [[Перейти к выводу]](#conclusion14). Подготовка данных к обучению, в том числе объединение или разделение необходимых данных в единый или различные дата-сеты, формирование новых признаков, дополнительных категорий и групп данных.
2. Шаг 2. Разработка моделей МО.
    - [[Перейти к разделу]](#step21). [[Перейти к выводу]](#conclusion21). Определение условий разработки моделей МО. Определиться с критериями разработки моделей, метриками качества моделей, критериями сравнения моделей.
    - [[Перейти к разделу]](#step22). [[Перейти к выводу]](#conclusion22). Основной этап моделирования. Разработать модели машинного обучения. Получить результаты для анализа их качества и скорости.
3. Шаг 3. [[Перейти к разделу]](#step3). Оценка качества работы моделей, выбор лучшей модели, оценка качества модели на тестовой выборке.

**[[Итоговый вывод]](#conclusion)**

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
import re
from tqdm import tqdm

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC

In [22]:
import lightgbm as lgb

In [ ]:
!python -m spacy download en_core_web_sm -q

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
import en_core_web_sm

In [ ]:
if spacy.__version__ != '3.2.6':
    print('Pls, restart the kernel')
    print(spacy.__version__)

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords

## Объявление констант и установок

In [ ]:
notebook_time_start = time.time()

In [ ]:
RANDOM_STATE = 546859

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
TEST_VALID_SIZE = 0.4

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
tqdm.pandas()

In [ ]:
BIG_PROCESS = False

## Шаг 1. Подготовка данных

### Загрузка и изучение данных 

<a id='step11' />

[Вернуться к началу](#begin)

In [ ]:
try:
    data = pd.read_csv(
        '../../datasets/toxic_comments.csv',
        index_col=0
    )
except (Exception) as e: 
    print(e)
    data = pd.read_csv(
        'https://code.s3.yandex.net/datasets/toxic_comments.csv',
        index_col=0
    )

Изучим общие сведения о данных

In [ ]:
data.info()

In [ ]:
data.head()

#### Вывод по разделу "Загрузка и изучение входных данных" 

<a id='conclusion11' />

[Вернуться к началу](#begin)

На основании общих сведений о входных данных таблицы `data` сделаны следующие выводы:

1. Данные содержат два столбца, один из которых cодержит комментарии для анализа, второй - целевой признак токсичности комментария.
2. На первый взгляд данные представлены на английском языке. Проверим это. 
3. Пропусков в данных не выявлено, целесообразно проверить данные на дубликаты без учета целевого признака.

### Преобработка данных 

<a id='step12' />

[Вернуться к началу](#begin)


Прозведем предобработку данных на основании ранее сделанных выводов:

1. Пропусков в данных не имеется.
2. Проверим наличие кириллических символов.
3. Проверим наличие дубликатов в столбце `text` и обработаем их при необходимости.
4. Обработаем типы данных.

Предварительно создадим копию входных данных для текущего этапа работы.

In [ ]:
preprocessing_data = data.copy()

Проверим наличие дубликатов в столбце `text`

In [ ]:
preprocessing_data.duplicated(subset='text').sum()

Дубликатов не имеется.

Напишем функцию, которая оставит в тексте только латинские символы, пробелы и знак апострофа. На вход она принимает текст, а возвращает очищенный текст. Дополнительно уберем лишние пробелы. Также приведем все слова к нижнему регистру.

In [ ]:
def clear_text(text):
    cleared_text = re.sub(r'[^a-zA-Z\'\❜]', ' ', text)
    cleared_text = re.sub(r'((\w)\2{2,})', ' ', cleared_text).split()   
    cleared_text = ' '.join(cleared_text).lower()
    return cleared_text

In [ ]:
preprocessing_data['cleared_text'] = preprocessing_data['text'].progress_apply(clear_text)

Напишем функцию, которая на вход принимает текст и возвращает лемматизированную строку.

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize(text):
    token = nltk.word_tokenize(text)
    text = [word for word in token]
    text = [lemmatizer.lemmatize(word,pos='v') for word in text]
    text = ' '.join(text)
    return text

In [ ]:
preprocessing_data['lemmatized_text'] = preprocessing_data['cleared_text'].progress_apply(lemmatize)

Проведем также лемматизацию с использованием библиотеки spaCy

In [ ]:
def lemmatize_spaCy(df, original='text'):
    if not BIG_PROCESS:
        try:
            spacy_lemmas = pd.read_csv('lemmatized_spaCy.csv', index_col=0)
            return spacy_lemmas['lemmatized_spaCy'].str.lower()
        except Exception as e:
            print(e)
    
    df = df.copy()
    nlp = en_core_web_sm.load()
    
    def lemmatize(text):
        lemmatized_text = nlp(text)
        lemmatized_text = (
            " ".join([token.lemma_ for token in lemmatized_text])
        )
        return lemmatized_text
    
    lemmatized = df[original].progress_apply(lemmatize)
    
    df['lemmatized_spaCy'] = lemmatized
    
    df['lemmatized_spaCy'].to_csv('lemmatized_spaCy.csv')
    
    return lemmatized

Однако, лематизация с использованием этого инструмента занимает слишком много времени. В связи с этим, загрузим готовый результат из файла, созданного локально

In [ ]:
preprocessing_data['lemmatized_text_spacy'] = lemmatize_spaCy(preprocessing_data, 'cleared_text')

In [ ]:
preprocessing_data.head(3)

Изучим результаты

In [ ]:
show_result = preprocessing_data.loc[[74584]]
#show_result = preprocessing_data.sample(1)
print("---> Идентификатор текста:\n", show_result.index)
print()
print("---> Пример исходного текста:\n", show_result.iloc[0,0])
print()
print("---> Пример очищенного текста:\n", show_result.iloc[0,2])
print()
print("---> Пример лемматизированного текста:\n", show_result.iloc[0,3])
print()
print("---> Пример лемматизированного текста через Spacy:\n", show_result.iloc[0,4])

В рассмотренном примере мы наблюдаем следующее:
1. Удалены не латинские символы. Даже в имени автора - José Luis Gómez Pérez. В таких ситуациях мы вместо слов получаем отдельные буквы. Целесообразно продумать вариант обработки таких случаев. В настоящей работе этого делать не будем.
2. При лемматизации с использованием nltk сокрашения типа I'm определяются верно, однако сокращения с not не могут быть лемматизированы.
3. При лемматизации с использованием spaCy этой проблемы не имеется.

Проверим на наличие пропусков после лемматизации.

In [ ]:
preprocessing_data[preprocessing_data['lemmatized_text_spacy'].isna()]

Все верно. Некоторые сообщения не содержать буквенных симоволов. Следовательно после очистки тексы были представлены как пустые. Удалим такие строки.

In [ ]:
preprocessing_data = preprocessing_data[~preprocessing_data['lemmatized_text_spacy'].isna()]

#### Вывод по разделу "Преобработка данных" 

<a id='conclusion12' />

[Вернуться к началу](#begin)


В ходе предобработки данных были достигнуты следующие результаты:

1. Пропусков и дубликатов в данных не обнаружено.
2. Произведена преобработка текстовых данных: токенизация, очистка и лемматизация. Лучший результат лемматизации показала библиотека spaCy.

Предобработка данных завершена, данные готовы к дальнейшему анализу.

### Исследовательский анализ данных 

<a id='step13' />

[Вернуться к началу](#begin)


В рамках исследовательского анализа данных оценим баланс классов нашей задачи.

In [ ]:
eda_data = preprocessing_data.copy()

In [ ]:
eda_data['toxic'].value_counts().plot(
    kind='pie', 
    autopct = '%1.0f%%',
    ylabel = '',
    title = 'Соотношение целевого признака. Столбец [toxic]',
)
plt.show()


#### Вывод по разделу "Исследовательский анализ данных" 

<a id='conclusion13' />

[Вернуться к началу](#begin)

В результате проведения исследовательского анализа было выявлено, что имеется существенный дисбаланс классов. Устранять его не будем, но учтем при делении на выборки (stratify) и при обучении модели (аттрибут class_weight='balanced').

### Подготовка данных 

<a id='step14' />

[Вернуться к началу](#begin)

В результате проведенной работы определено, что в работе необходимо решить задачу бинарной классификации для текстов с применением техник NLP. Для формирования модели необходимо сгенерировать признаки путем векторизации предобработанных текстов. Перед векторизацией необходимо разбить данные на выборки: обучающую, валидационную и тестовую; в связи с тем, что первичную векторизацию необходимо реализовавыть на обучающей выборке и затем алгоритмы векторизации использовать на валидационной и тестовой выборках.

Предварительно создадим копию входных данных для текущего этапа работы.

In [ ]:
process_data = eda_data.copy()
process_data.head()

Разделим данные на обучающую и тестовую выборки в отношении 0.6:0.4

In [ ]:
X = process_data
y = process_data['toxic']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_VALID_SIZE, random_state=RANDOM_STATE, stratify=y)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Создадим функцию обучения, кросс-валидации и тестирования

In [ ]:
def get_score(model, vectorizer, 
              X_train, column, y_train,
              X_test=None, y_test=None):
    model = make_pipeline(vectorizer, model)
    if X_test is None or y_test is None:
        score = cross_val_score(model, X_train[column], y_train,
                               scoring='f1', cv=3, n_jobs=6).mean()
    else:
        model.fit(X_train[column], y_train)
        y_pred = model.predict(X_test[column])
        score = f1_score(y_test, y_pred)
    return score

#### Вывод по разделу "Подготовка данных" 

<a id='conclusion14' />

[Вернуться к началу](#begin)

Пайплайн подготовлен с учетом принципов работы с текстами:
1. На вход подается векторайзер для формирования векторов текста.
2. На вход подается только один столбец с текстами.
3. Применена кросс-валидация, по которой будем оценивать качество модели.
4. При передаче в функцию тестовых данных, она вернет результирующую оценку для тестовой выборки.

В ходе работы также подготовлены тренировочная и тестовая выборки. Данные тестовой выборки будут использованы только в конце работы.

## Шаг 2. Разработка моделей МО


### Определение условий разработки моделей МО 

<a id='step21' />

[Вернуться к началу](#begin)


**Постановка задачи**

Заказчиком поставлена задача бинарной классификации с применением техник NLP.

**Выбор метрики**

В задачах бинарной классификации используются различные метрики. В связи с тем, что в решаемой задаче нет необходимости минимизировать ошибку I или II рода, целесообразно использовать обобщенную метрику. Таковой является F1-мера.

**Необходимый уровень метрики**

 Заказчик установил в качестве критерия качества моделей указанную метрику в уровне не менее 0,75 на тестовой выборке.

**План основного этапа моделирования**

На основании сделанных выводов и поставленных задач на основном этапе моделирования выполним следующие шаги:
1. Доработаем входные данные, исключив стоп-слова, не несущие смысловой нагрузки.
2. Используем два векторайзера для получения векторов текстов: CountVectorizer() и TfidfVectorizer().
3. Построим три модели для сравнения: LogisticRegression(), LinearSVC(), LightGBM.
4. Получим результаты кросс-валидации трех моделей для каждого векторайзера.
5. Кроме того, произведем построение моделей для двух видом лемматизации текстов: с использованием WordNet и spaCy.


#### Вывод по разделу "Определение условий разработки моделей МО" 

<a id='conclusion21' />

[Вернуться к началу](#begin)

В результате анализа целей и задач работы определены условия разработки моделей МО и сформирован план основного этапа моделирования.


### Основной этап моделирования 

<a id='step22' />

[Вернуться к началу](#begin)

Выполним шаги в соответствии с планом, определенным в предыдущем шаге.

In [ ]:
stopwords = list(nltk_stopwords.words('english'))

Для выполнения шагов сформируем словари с моделями, векторазерами и столбцами для обучения. Затем переберем их в циклах, передавая значения в созданную ранее функцию

In [ ]:
models = {
    'LogisticRegression()': LogisticRegression(max_iter=1000),
    'LinearSVC()': LinearSVC(
        dual=False, 
        random_state=RANDOM_STATE, 
        max_iter=1000, 
        class_weight='balanced', 
        C=0.5
    ),
    'LightGBM': lgb.LGBMClassifier(
        random_state=RANDOM_STATE, 
        class_weight='balanced', 
        n_estimators=100
    )
}

In [ ]:
vectorizers = {
    'CountVectorizer': CountVectorizer(stop_words='english',
                                       dtype=np.float32),
    'TfidfVectorizer': TfidfVectorizer(stop_words=stopwords,
                                       dtype=np.float32),
}

In [ ]:
columns = [
    'lemmatized_text',
    'lemmatized_text_spacy'
]

In [ ]:
calculate_time = time.time()

for column in columns:
    for model_name in models:
        for vectorizer_name in vectorizers:
            score = get_score(models[model_name],
                vectorizers[vectorizer_name],
                X_train=X_train,
                y_train=y_train,
                column=column)
            stage_time = time.strftime("%M минут %S секунд",
                                       time.gmtime(
                                           time.time() - calculate_time
                                       ))
            print(model_name, 'with', vectorizer_name, 
                  'for column', column, '\n   --> CV f1_score:', 
                  round(score, 3),
                  '--- Время выполнения:', stage_time)
            calculate_time = time.time()

Для уточнения наших моделей имеет смысл осуществить подбор некоторых гиперпараметров для сформированных моделей. Для LightGBM целесообразно использовать большее количество деревьев, однако, время обучения будет значительно выше. 
Попробуем подобрать гиперпараметры логистической регрессии. Указанная модель обучается довольно быстро, гиперпараметров для нее не очень много. Попробуем произвести поиск по сетке.

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipe = Pipeline(
    [
        ('vectorizer', None),
        ('model', None)
    ]
)

In [ ]:
param_grid = [
    {
        'vectorizer': [
            CountVectorizer(stop_words='english',
                            dtype=np.float32),
            TfidfVectorizer(stop_words=stopwords,
                            dtype=np.float32),
        ],
        'model': [LogisticRegression(max_iter=1000)],
        'model__penalty': ['l1', 'l2'],
        'model__C': range(5, 15),        
    }
]

In [ ]:
gs = GridSearchCV(
    pipe, 
    param_grid, 
    n_jobs=-1,
    cv=3,
    verbose=10,
    scoring='f1'
)

In [ ]:
%%time

gs.fit(X_train['lemmatized_text_spacy'], y_train)

In [ ]:
print(gs.best_estimator_)

In [ ]:
print(gs.best_score_)

In [ ]:
print(gs.best_params_)

#### Вывод по разделу "Основной этап моделирования" 

<a id='conclusion22' />

[Вернуться к началу](#begin)

В ходе работы в соответствии с определенным планом достигнуты следующие результаты:

1. Были обучены три модели с двумя векторайзерами для двух лемматизированных столбцом и оценена метрика.
2. LinearSVC() единственная показала результаты выше порогового уровня на обучающей выборке. 
3. Лучшие результаты по итогам кросс-валидации показывает модель LinearSVC() с метрикой f1=0.768. Векторайзер - CountVectorizer, столбец - 'lemmatized_text_spacy'.

В результате гиперпараметров для модели логистической регрессии удалось увеличить качество модели (на основании кросс-валидации). Примем лучшей именно модель логистической регрессии с параметрами: C=10 и penalty='l2' для векторайзера TF-IDF.

## Шаг 3. Оценка качества работы моделей, выбор лучшей модели

### Анализ разработанных моделей МО 

<a id='step3' />

[Вернуться к началу](#begin)

В связи с тем, что основным критерием выбора модели является ее метрика f1. То лучшей моделью признается LightGBM. Оценим ее качество на тествой выборке.

In [ ]:
score = get_score(
    lgb.LGBMClassifier(
        random_state=RANDOM_STATE,
        class_weight='balanced',
        max_depth=-1,
        num_leaves=31,
        n_estimators=1000
    ),
    CountVectorizer(stop_words='english', dtype=np.float32),
    X_train=X_train,
    y_train=y_train,
    column='lemmatized_text_spacy',
    X_test=X_test,
    y_test=y_test,)
score

In [ ]:
score = get_score(
    LogisticRegression(max_iter=1000,
                      C=10, penalty='l2'),
    TfidfVectorizer(stop_words=stopwords, dtype=np.float32),
    X_train=X_train,
    y_train=y_train,
    column='lemmatized_text_spacy',
    X_test=X_test,
    y_test=y_test,)
score

Необходимый результат на тестовой выборке достигнут.

## Итоговый вывод  

<a id='conclusion' />

[Вернуться к началу](#begin)

В соответствии с планом работы были достигнуты следующие результаты.

**А. Изучение входных данных**

Для проведения исследования была получена выборка из 159292 записей. Данные содержат два столбца, один из которых cодержит тексты комментариев, второй - целевой признак (отметку) о токсичности комментариев.

**Б. Предобработка данных**

В ходе предобработки данных:

1. Пропусков и дубликатов в данных не обнаружено.
2. Произведена преобработка текстовых данных: токенизация, очистка и лемматизация с использованием WordNet и spaCy. Лучший результат лемматизации показала библиотека spaCy.

**В. Результаты исследовательского анализа**

В результате проведения исследовательского анализа было выявлено, что имеется существенный дисбаланс классов. Принято решение об учете этой особенности при моделировании (при делении на выборки (stratify) и при обучении модели (аттрибут class_weight='balanced')).

**Г. Подготовка данных**

Для дальнейшего обучения моделей МО выполнены следующие работы:
- разработаны средства автоматизации процесса обучения моделей (pipeline) с учетом результатов предобработки данных;
- подготовлены выборки для обучения моделей соотношении 60% тренировочной выборки к 40% тестовой выборки.

**Д. Разработка моделей машинного обучения**

В результате проведенной работы по обучению моделей достигнуты следующие результаты:
1. Были обучены три модели с двумя векторайзерами для двух лемматизированных столбцом и оценена метрика.
2. LinearSVC() и LightGBM показали результаты выше порогового уровня на обучающей выборке. 
3. Лучшие результаты по итогам кросс-валидации показывает модель градиентного бустинга с метрикой f1=0.768 при векторайзере CountVectorizer для текстов, лемматизированных с использованием spaCy.

**Е. Анализ разработанных моделей МО**

В связи с тем, что критерием выбора модели является ее метрика f1. То лучшей моделью признана LightGBM. 

**Ж. Оценка качества модели на тестовой выборке**

В результате проведенной работы лучшая модель для заказчика - LightGBM, которая показала качество на тестовой выборе: f1=0.780, что выше установленного критерия заказчика.

**З. Достижение цели работы**

Построена модель МО, которая предсказывает токсичность комментариев с точностью выше определенной требованиями заказчика. Цель работы достигнута.

**И. Перспективы развития**

Построенная модель показывает среднее качество. Возможно достижение более высоких результатов:
1. При подборе гиперпараметров для лучшей модели и отстающих.
2. При использовании предобученной модели BERT.

Оба варианта развития моделей требуют больших вычислительных мощностей.

In [ ]:
notebook_time = time.time() - notebook_time_start
print(
    'Общее время выполнения: ',
    time.strftime("%H часов %M минут %S секунд", time.gmtime(notebook_time))
)